# Input data
## AWS Credentials
AWS credentials are by convention stored in `~/.aws/credentials` text file. In this file section `[default]` contains two key-value pairs in format: `key = value`. Values for `aws_access_key_id` and `aws_secret_access_key` are required to connect with s3 storage.

Example, minimal `~/.aws/credentials` file looks as follows:
```
[default]
aws_access_key_id = ASDFQWER
aws_secret_access_key = abcdefghijk12345678
```

In [1]:
import configparser
import os.path

config_parser = configparser.ConfigParser()
config_parser.read_file(open(os.path.join(os.path.expanduser('~'), '.aws', 'credentials')))
config_parser.sections()

aws_access_key_id = config_parser.get('default', 'AWS_ACCESS_KEY_ID')
aws_secret_access_key = config_parser.get('default', 'AWS_SECRET_ACCESS_KEY')

## S3 connection
For accessing input data [cottoncandy](https://github.com/gallantlab/cottoncandy) project is used. It provides a usefull abstraction for storing and accessing numpy data arrays in s3 storage.

In [3]:
import cottoncandy
cci = cottoncandy.get_interface('alpenglowoptics', 
                                ACCESS_KEY=aws_access_key_id, 
                                SECRET_KEY=aws_secret_access_key,
                                endpoint_url='https://s3.amazonaws.com',
                                verbose=False)

## Defining source file paths
This notebook is ment to run stiching algorithm on an example test case. Following code defines directory structure for input files of this single test case.

In [4]:
files_per_strip = 1800
number_of_strips = 3
path_format = '/phantom_2_7_2017/{strip_id:06d}/{strip_id:06d}_{strip_file_id:05d}.tif'

source_files = [path_format.format(strip_id=strip_id, strip_file_id=strip_file_id) for strip_id in range(number_of_strips) for strip_file_id in range(1, files_per_strip + 1) ]

# Sampling
Sampling consists of checking dimensions of images stored in input files and their pixcel type (color depth).  

In [5]:
import alpenglow.files as af

sample_image = af.s3_to_array(source_files[0], cci)
shape = sample_image.shape
pixcel_type = sample_image.dtype

# Processing
Processing is simplified. It reads only two first strips and runs one stiching phase.

In [6]:
stiched_stack = af.read_strip_files(source_files, files_per_strip, number_of_strips - 1, cci, pixcel_type, shape)
for ss in range(number_of_strips - 2, -1, -1):
    strip = af.read_strip_files(source_files, files_per_strip, ss, cci, pixcel_type, shape)
    stiched_stack = ast.stitch_zstack(stiched_stack, strip)

NameError: name 'ast' is not defined

# Displaying results
Stiched result image can be ploted using matplotlib.

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(stiched_stack[256])
plt.show()